# The Model

In [6]:
DecayRate = 1 # The number of activation units lost every iteration (from each node)
ActivationStrength = 0.1 # The relation between unit activation and the activation it exerts through excitatory connections
InhibitionStrength = 0.1 # The relation between unit activation and the inhibition it exerts through inhibitory connections

max_iter = 100 # Maximum number of iterations (per trial)
ActivationThreshold = 100 # Threshold activation level for a lower-level unit to be cosidered the final interpratation


class CognitiveControl(object):
    def __init__(self,BiasingMult=0.2):
        self.ConflictMonitoring = 0
        self.Biasing = 0
        self.BiasingMult = BiasingMult #....
 
    def Update(self,LingKnow,Stoop):
        # increase CM activity based on ratio between SubjIsAgent & SubjIsTheme
        
        # increase CM activity based on ratio between Blue & Red

        # increase B activation based on CM activation
                
        # decay
        self.ConflictMonitoring -= DecayRate
        self.Biasing -= DecayRate      
        # prevent negative activation values
        if self.ConflictMonitoring < 0:
            self.ConflictMonitoring = 0
        if self.Biasing < 0:
            self.Biasing = 0

            
class LinguisticKnowledge(object):
    def __init__(self, WorldAct=0, MorphSyntAct=0):
        # Higher-Level
        self.WorldKnowledge = WorldAct
        self.MorphoSyntacticKnowledge = MorphSyntAct
        # Lower-Level
        self.SubjIsAgent = 0
        self.SubjIsTheme = 0
        
    def InputAct(self,WK,MSK):
        self.WorldKnowledge += WK
        self.MorphoSyntacticKnowledge += MSK

    def Update(self,CogCtrl,Stoop):        
        # save initial activations for calculations (to make sure all updating happens "at once")
        WorldKnowledgeInitialAct = self.WorldKnowledge
        MorphoSyntacticKnowledgeInitialAct = self.MorphoSyntacticKnowledge
        SubjIsAgentInitialAct = self.SubjIsAgent
        SubjIsThemeInitialAct = self.SubjIsTheme

        # increase WK & MSK activity based on CogCtrl.Biasing
        self.WorldKnowledge += (CogCtrl.Biasing*CogCtrl.BiasingMult)
        self.MorphoSyntacticKnowledge += (CogCtrl.Biasing*CogCtrl.BiasingMult)        
        
        # increase SubjIsAgent & SubjIsTheme based on WK & MSK
        self.SubjIsAgent += (MorphoSyntacticKnowledge*ActivationStrength) # SubjIsAgent is supported by WK
        self.SubjIsTheme += (WorldKnowledgeInitialAct*ActivationStrength) # SubjIsTheme is supported by MSK
        
        # lateral inhibition between SubjIsAgent & SubjIsTheme
        self.SubjIsAgent -= (self.SubjIsAgentInitialAct*InhibitionStrength)
        self.SubjIsTheme -= (SubjIsAgentInitialAct*InhibitionStrength)
        
        # decay
        self.WorldKnowledge -= DecayRate
        self.MorphoSyntacticKnowledge -= DecayRate
        self.SubjIsAgent -= DecayRate
        self.SubjIsTheme -= DecayRate
        # prevent negative activation values
        if self.WorldKnowledge < 0 :
            self.WorldKnowledge = 0
        if self.MorphoSyntacticKnowledge <0:
            self.MorphoSyntacticKnowledge = 0
        if self.SubjIsAgent < 0:
            self.SubjIsAgent = 0
        if self.SubjIsTheme < 0:
            self.SubjIsTheme = 0

        
class StroopTaskRepresentation(object):
    def __init__(self, TextAct=0, FontColAct=0):
        # Higher-Level
        self.Text = TextAct
        self.FontColor = FonrColAct
        # Lower-Level
        self.Blue = 0
        self.Red = 0
            
    def InputAct(self,Text,FontColor):
        self.Text += Text
        self.FontColor += FontColor
        

    def Update(self,CogCtrl,LingKnow):
        # save initial activations for calculations (to make sure all updating happens "at once")
        TextInitialAct = self.Text
        FontColorInitialAct = self.FontColor
        BlueInitialAct = self.Blue
        RedInitialAct = self.Red
        
        # increase Text & FontColor activity based on CogCtrl.Biasing
        self.Text += (CogCtrl.Biasing*CogCtrl.BiasingMult)
        self.FontColor += (CogCtrl.Biasing*CogCtrl.BiasingMult)

        # increase Blue & Red based activity based on Text & FontColor
        self.Blue += (FontColorInitialAct*ActivationStrength) # Blue is supported by FontColor
        self.Red +- (TextInitialAct*ActivationStrength) # Red is supported by Text

        # lateral inhibition between Blue & Red
        self.Blue -= (RedInitialAct*InhibitionStrength)
        self.Red -= (BlueInitialAct*InhibitionStrength)
        
        # decay
        self.Text -= TextAct
        self.FontColor -= DecayRate
        self.Blue -= DecayRate
        self.Red -= DecayRate
        # prevent negative activation values
        if self.Text < 0:
            self.Text = 0
        if self.FontColor <0:
            self.FontColor = 0
        if self.Blue < 0:
            self.Blue = 0
        if self.Red < 0:
            self.Red = 0

        

def UpdateAll(CogCtrl,LingKnow,Stroop):    
    CogCtrl_prev = copy.copy(CogCtrl)
    LingKnow_prev = copy.copy(LingKnow)
    Stroop_prev = copy.copy(Stroop)

    CogCtrl.Update(LingKnow_prev,Stroop_prev)
    LingKnow.Update(CogCtrl_prev,Stroop_prev)
    Stroop.Update(CogCtrl_prev,LingKnow_prev)
    return [CogCtrl, LingKnow, Stroop]


def RunTrial(InputAct:tuple,CogCtrl=None,LingKnow=None,Stroop=None):
    '''
    InputAct specifies the input activations for WK,MSK,Text,FontColor (in that order).
    '''
    i = 1
    
    if CogCtrl == None:
        CogCtrl = CognitiveControl()
    if LingKnow == None:
        LingKnow = LinguisticKnowledge()
    if Stroop == None:
        Stroop = StroopTaskRepresentation()

    LingKnow.InputAct(InputAct[0],InputAct[1])
    Stroop.InputAct(InputAct[2],InputAct[3])

    while i <= max_iter:
        UpdateAll(CogCtrl,LingKnow,Stroop)
        i += 1
        MaxAct = max(LingKnow.SubjIsAgent,LingKnow.SubjIsTheme,Stroop.Blue,Stroop.Red)
        if MaxAct > ActivationThreshold:
            break
    
    # determine the final interpratation
    if [LingKnow.SubjIsAgent,LingKnow.SubjIsTheme,Stroop.Blue,Stroop.Red].count(MaxAct) > 1:
        Winner = None
    if LingKnow.SubjIsAgent == MaxAct:
        Winner = LingKnow.SubjIsAgent
    if LingKnow.SubjIsTheme == MaxAct:
        Winner = LingKnow.SubjIsTheme
    if Stroop.Blue == MaxAct:
        Winner = Stroop.Blue
    if Stroop.Red == MaxAct:
        Winner = Stroop.Red
    
    return i, Winner, CogCtrl, LingKnow, Stroop


def RunTrialSequence(Trials:"list of tuples"):
    '''
    Trials is a list of tuples.
    Each tuple consists of 4 values that are the input activations of WK,MSK,Text,FontColor (in that order) in a specific trial.
    '''
    CogCtrl = CognitiveControl()
    LingKnow = LinguisticKnowledge()
    Stroop = StroopTaskRepresentation()
    
    Results = []
    for Trial in Trials:
        i, Winner, CogCtrl, LingKnow, Stroop = RunTrial(Trial,CogCtrl,LingKnow,Stroop)
        Results.append([i,Winner])
        
    return Results

# Cognitive control makes conflict resolution faster

## In Stroop

In [ ]:
# Congruent Stroop trial with BiasingMult = 0

# Congruent Stroop trial with BiasingMult = 0.2

# Inongruent Stroop trial with BiasingMult = 0

# Incongruent Stroop trial with BiasingMult = 0.2

# Figure

Cognitive control makes incongruent Stroop trials faster (and does not affect congruent trials).

## In Language

In [ ]:
# Control sentence trial with BiasingMult = 0

# control sentence trial with BiasingMult = 0.2

# Anomalous sentence trial with BiasingMult = 0

# Anomalous sentence trial with BiasingMult = 0.2

# Figure

The same cognitive control system also makes trials with linguistic conflict faster (and does not affect control sentences).

# Cross-Task Adaptation

In [ ]:
# A Congruent Stroop -> Control Sentence sequence

# An Inongruent Stroop -> Control Sentence sequence

# A Congruent Stroop -> Anomalous Sentence sequence

# An Incongruent Stroop -> Anomalous Sentence sequence

# Figure

The engagment of cognitive control in an Incongruent Stroop Trial makes the subsequent Anomalous Sentence trial faster (but does not affect Control Sentences).

# Possible Additional Figures
Activation levels of a node throughout a trial - this information is currently not saved. Needs to be changed if we want that.